In [ ]:
# coding=utf-8
# Copyright 2023 Frank Latos AC8P
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
from necutil import nec5_sim_stdio3, plot_complex_z, plot_azimuth2, plot_elevation, plot_vswr, in2m, plot_azimuth_thumb
from necutil import gen_tapered_el2

import numpy as np, time

# A simple 3-el Yagi with uniform 0.5" elements
yagi = '\n'.join(
    ['CE 3-el Yagi',               
     'GW 1 20 -0.91 -2.79 0 -0.91 2.79 0 0.00635',          # R
     'GW 2 20 0 -2.56 0 0 2.56 0 0.00635',                  # DE
     'GW 3 20 1.37 -2.44 0 1.37 2.44 0 0.00635',            # D
     'GE 0 0',                                              # End of geometry; no ground plane specified
     'EX 4 2 10 2 1.0 0.0',                                 # Excitation: current source (1A), tag=2, segment=10, far end=2
     'FR 0 9 0 0 28.0 0.125',                               # Frequencies for XQ card: 28.0 - 29.0 MHz
     'XQ 0',                                                # Simulate feedpoint impedance
     'FR 0 1 0 0 28.5 0',                                   # Frequency for RP card: 28.5 MHz
     'RP 0 19 37 0000 0 0 5 5',                             # theta: 0-90, phi: 0-180 (5 deg grid)
     'EN\n'                                                 # End
    ])



In [2]:
%%timeit
result = nec5_sim_stdio3([yagi]*100)
# My exec time = 2.56s --> 25.6 ms/simulation

2.53 s ± 60.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
# A simple 3-el Yagi with uniform 0.5" elements, MININEC ground, z=10m
yagi_mininec_gnd = '\n'.join(
    ['CE 3-el Yagi',               
     'GW 1 20 -0.91 -2.79 10 -0.91 2.79 10 0.00635',        # R
     'GW 2 20 0 -2.56 10 0 2.56 10 0.00635',                # DE
     'GW 3 20 1.37 -2.44 10 1.37 2.44 10 0.00635',          # D
     'GE 1 0',                                              # End of geometry; ground plane specified
     'GD 0 0 0 0 13 0.005 0 0',                             # Some typical MININEC ground parameters
     'EX 4 2 10 2 1.0 0.0',                                 # Excitation: current source (1A), tag=2, segment=10, far end=2
     'FR 0 9 0 0 28.0 0.125',                               # Frequencies for XQ card: 28.0 - 29.0 MHz
     'XQ 0',                                                # Simulate feedpoint impedance
     'FR 0 1 0 0 28.5 0',                                   # Frequency for RP card: 28.5 MHz
     'RP 0 19 37 0000 0 0 5 5',                             # theta: 0-90, phi: 0-180 (5 deg grid)
     'EN\n'                                                 # End
    ])


In [9]:
%%timeit
result = nec5_sim_stdio3([yagi_mininec_gnd]*100)
# My exec time = 3.6s --> 36.0 ms/simulation

3.63 s ± 34.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
# A simple 3-el Yagi with uniform 0.5" elements, Sommerfeld ground, z=10m
yagi_somm_gnd = '\n'.join(
    ['CE 3-el Yagi',               
     'GW 1 20 -0.91 -2.79 10 -0.91 2.79 10 0.00635',        # R
     'GW 2 20 0 -2.56 10 0 2.56 10 0.00635',                # DE
     'GW 3 20 1.37 -2.44 10 1.37 2.44 10 0.00635',          # D
     'GE 1 0',                                              # End of geometry; ground plane specified
     'GN 0 0 0 0 13 0.005 1 0 SOM_285_10.nex',              # Sommerfield ground
     'EX 4 2 10 2 1.0 0.0',                                 # Excitation: current source (1A), tag=2, segment=10, far end=2
     'FR 0 9 0 0 28.0 0.125',                               # Frequencies for XQ card: 28.0 - 29.0 MHz
     'XQ 0',                                                # Simulate feedpoint impedance
     'FR 0 1 0 0 28.5 0',                                   # Frequency for RP card: 28.5 MHz
     'RP 0 19 37 0000 0 0 5 5',                             # theta: 0-90, phi: 0-180 (5 deg grid)
     'EN\n'                                                 # End
    ])

# Run one first to generate Sommerfeld ground tables to disk
_ = nec5_sim_stdio3([yagi_somm_gnd])

In [11]:
%%timeit
result = nec5_sim_stdio3([yagi_somm_gnd]*100)
# My exec time = 10.3s --> 103.0 ms/simulation
# Also tried 'GN 1' perfect ground:  My exec time = 2.34s --> 23.4 ms/simulation

10.1 s ± 895 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
result = nec5_sim_stdio3([yagi]*100)

xl_max = 200                    # Max inductive matching reactance considered (ohms)
z0 = 50
x_m = np.arange(1,xl_max+1,dtype=float)             # Matching reactance values considered, e.g. 1,2,3...200 ohms
# Pre-calculate 1/Xl for range of matching inductive reactances considered
freqs = np.linspace(28.0, 29.0, num=9)        # Freqs used in feedpoint z simulation
# Array of (1 / matching reactance) at each freq; shape = (len(x_m), len(freqs))
Xlmr = np.reciprocal(((1.0j)*x_m)[:,None] @ (freqs[None,:] / 28.5))         # L

def find_opt_match(x):    
    rzs = np.reciprocal(x)                                          # 1/z
    zmatch = np.reciprocal(Xlmr + rzs)                              # 1 / ( 1/z + 1/Xl )
    abs_refl_coef = np.abs((zmatch-z0) / (zmatch+z0))               # Reflection coefs
    vswr_curves = (1 + abs_refl_coef) / (1 - abs_refl_coef)         # vswr
    max_vswr = np.max(vswr_curves, axis=1)                          # Max vswr for any row (reactance value)
    idx = np.argmin(max_vswr)                                       # Index of min vswr value
    # Returns:  [0]     optimum matching inductive reactance
    #           [1]     max vswr within band of interest
    #           [2:]    vswr at each freq in band (# of points = f_num = 9 in this example)
    return np.hstack([x_m[idx], max_vswr[idx], vswr_curves[idx]])


In [13]:
%%timeit
# Extracts feedpoint complex z for each design --> complex array of shape (#designs, #freqs)
zs = np.array([[freq[1] for freq in des[0][0]] for des in result])

# Compute optimum matching reactance, max vswr within band for each design
xl_mv = np.apply_along_axis(find_opt_match, 1, zs)
vswr = xl_mv[:,1][:,None]                   # Max vswr for each design
opt_xl = xl_mv[:,0][:,None]                 # Corresponding matching reactance
vswr_curves = xl_mv[:,2:]                   # VSWR at each freq in band (f_num)

# My exec time = 19.2ms --> 0.192 ms/simulation


20.8 ms ± 70.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [2]:
from necutil import convert_rp_array, convert_rp_array_0, integrate_power_density_3
result = nec5_sim_stdio3([yagi]*100)

In [5]:
from numba import jit,njit,float64
import numba as nb

# @njit
# def get_angle_ranges(grid, startang, endang):
#     ngrid = len(grid)
#     halfgrid = (grid[1] - grid[0]) / 2
#     angs = np.zeros((2,ngrid))
#     angs[0,:] = grid-halfgrid
#     angs[1,:] = grid+halfgrid
#     angs = np.minimum(np.maximum(angs,startang), endang)
#     return angs

@njit
def integrate_power_density_4(arr, thetal=0.0, thetah=90.0, phil=0.0, phih=180.0):

    # Compute span of angles associated with each grid point of radiation pattern
    #   grid    angles on the grid calculated by RP card, e.g. [0,10,20,...,180] or [0,10,20,...,90]
    #   startang, endang    range of interest (any float value)
    def get_angle_ranges(grid, startang, endang):
        ngrid = len(grid)
        halfgrid = (grid[1] - grid[0]) / 2
        angs = np.zeros((2,ngrid))
        angs[0,:] = grid-halfgrid
        angs[1,:] = grid+halfgrid
        angs = np.minimum(np.maximum(angs,startang), endang)
        return angs

    assert arr.shape[0] in [91, 46, 31, 19, 16, 11, 10, 7, 6, 4, 3]
    assert arr.shape[1] in [181, 91, 61, 46, 37, 31, 21, 19, 16, 13, 11, 10, 7, 5, 3]
    thinc = 90 // (arr.shape[0] - 1)            # Grid increment (theta)
    phinc = 180 // (arr.shape[1] - 1)           # Grid increment (phi)
    thgrid = np.arange(0,91,thinc)        # e.g. [0, 10, ..., 90] for grid = 10
    phgrid = np.arange(0,181,phinc)       #      [0, 10, ..., 180]

    # 'tweight' = areas of horizontal strips associated with each grid point (within thetal,thetah limits)
    thlimits = np.cos(np.deg2rad(get_angle_ranges(thgrid, thetal, thetah)))         # Angle range to integrate for each grid point
    tweight = np.pi * (thlimits[0,:] - thlimits[1,:])

    # 'pweight' = frac of total area represented by vertical strips associated with each grid point (within phil,phih limits)
    phlimits = get_angle_ranges(phgrid, phil, phih)         # Angle range to integrate for each grid point
    pweight = (phlimits[1,:] - phlimits[0,:]) / 180
    
    # Sum area and power weighted by area
    #   Convert dB power array to ratio, mult by corresponding area 
    #    -> pweight broadcast along cols, weight[:,None] is a column vec, broadcast along rows
    weighted_pwr = tweight[:,None] * (pweight * np.power(10, arr / 10))      # Mult by weight (as col vec)
    # Total relative power
    pwr = np.sum(weighted_pwr)
    # Area of selected region
    area = np.sum(tweight) * np.sum(pweight)

    return pwr, area


In [7]:
# %%timeit
# Create list of rad pat arrays suitable for integrate_power_density_3()

# Original: convert_rp_array_0() (non-numba)
# My exec time = 4.98 ms --> 0.049 ms/simulation

# New numba version: convert_rp_array()
# My exec time = 3.37 ms --> 0.033 ms/simulation

rps = [convert_rp_array(res[1][0][0][1]) for res in result]           

In [10]:
%%timeit
from necutil import integrate_power_density_4
# Integrate response over some range of theta, phi

# Original version integrate_power_density_3()   (non-numba)
# My exec time = 369 ms --> 3.69 ms/simulation
# pwr = [integrate_power_density_3(arr, thetal=0.0, thetah=90.0, phil=90.0, phih=180.0) for arr in rps]

# Rewritten using numba: integrate_power_density_4()
# My exec time = 3 ms --> 30 us/simulation (!!)
pwr = [integrate_power_density_4(arr, thetal=0.0, thetah=90.0, phil=90.0, phih=180.0) for arr in rps]

2.89 ms ± 282 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
from necutil import integrate_power_density_4
integrate_power_density_4(rps[0], thetal=70.0, thetah=90.0, phil=90.0, phih=91.0)


(0.00011074625880102366, 0.005969377609175829)

In [3]:
from numba import jit,njit,float64
import numba as nb

# Generate some results to use for testing
from necutil import nec5_sim_stdio3, convert_rp_array
yagi = '\n'.join(
    ['CE 3-el Yagi',               
     'GW 1 20 -0.91 -2.79 0 -0.91 2.79 0 0.00635',          # R
     'GW 2 20 0 -2.56 0 0 2.56 0 0.00635',                  # DE
     'GW 3 20 1.37 -2.44 0 1.37 2.44 0 0.00635',            # D
     'GE 0 0',                                              # End of geometry; no ground plane specified
     'EX 4 2 10 2 1.0 0.0',                                 # Excitation: current source (1A), tag=2, segment=10, far end=2
     'FR 0 9 0 0 28.0 0.125',                               # Frequencies for XQ card: 28.0 - 29.0 MHz
     'XQ 0',                                                # Simulate feedpoint impedance
     'FR 0 1 0 0 28.5 0',                                   # Frequency for RP card: 28.5 MHz
     'RP 0 19 37 0000 0 0 5 5',                             # theta: 0-90, phi: 0-180 (5 deg grid)
     'EN\n'                                                 # End
    ])
result = nec5_sim_stdio3([yagi]*100)
arr180 = convert_rp_array(result[0][1][0][0][1])

yagi = '\n'.join(
    ['CE 3-el Yagi',               
     'GW 1 20 -0.91 -2.79 0 -0.91 2.79 0 0.00635',          # R
     'GW 2 20 0 -2.56 0 0 2.56 0 0.00635',                  # DE
     'GW 3 20 1.37 -2.44 0 1.37 2.44 0 0.00635',            # D
     'GE 0 0',                                              # End of geometry; no ground plane specified
     'EX 4 2 10 2 1.0 0.0',                                 # Excitation: current source (1A), tag=2, segment=10, far end=2
     'FR 0 9 0 0 28.0 0.125',                               # Frequencies for XQ card: 28.0 - 29.0 MHz
     'XQ 0',                                                # Simulate feedpoint impedance
     'FR 0 1 0 0 28.5 0',                                   # Frequency for RP card: 28.5 MHz
     'RP 0 19 72 0000 0 0 5 5',                             # theta: 0-90, phi: 0-355 (5 deg grid)
     'EN\n'                                                 # End
    ])
result = nec5_sim_stdio3([yagi]*100)
arr360 = convert_rp_array(result[0][1][0][0][1])

# Compute span of angles associated with each grid point of radiation pattern
#   grid    angles on the grid calculated by RP card, e.g. [0,10,20,...,180] or [0,10,20,...,90]
#   startang, endang    range of interest (any float value)
@njit
def get_angle_ranges(grid, startang, endang):
    ngrid = len(grid)
    halfgrid = (grid[1] - grid[0]) / 2
    angs = np.zeros((2,ngrid))
    angs[0,:] = grid-halfgrid
    angs[1,:] = grid+halfgrid
    angs = np.minimum(np.maximum(angs,startang), endang)
    return angs
@njit
def integrate_power_density_4(arr, thetal=0.0, thetah=90.0, phil=0.0, phih=180.0):
    assert arr.shape[0] in [91, 46, 31, 19, 16, 11, 10, 7, 6, 4, 3, 2]
    assert arr.shape[1] in [181, 91, 61, 46, 37, 31, 21, 19, 16, 13, 11, 10, 7, 6, 5, 4, 3, 2]
    thinc = 90 // (arr.shape[0] - 1)            # Grid increment (theta)
    phinc = 180 // (arr.shape[1] - 1)           # Grid increment (phi)
    thgrid = np.arange(0,91,thinc)        # e.g. [0, 10, ..., 90] for grid = 10
    phgrid = np.arange(0,181,phinc)       #      [0, 10, ..., 180]

    # 'tweight' = areas of horizontal strips associated with each grid point (within thetal,thetah limits)
    thlimits = np.cos(np.deg2rad(get_angle_ranges(thgrid, thetal, thetah)))         # Angle range to integrate for each grid point
    tweight = np.pi * (thlimits[0,:] - thlimits[1,:])

    # 'pweight' = frac of total area represented by vertical strips associated with each grid point (within phil,phih limits)
    phlimits = get_angle_ranges(phgrid, phil, phih)         # Angle range to integrate for each grid point
    pweight = (phlimits[1,:] - phlimits[0,:]) / 180
    
    # Sum area and power weighted by area
    #   Convert dB power array to ratio, mult by corresponding area 
    #    -> pweight broadcast along cols, weight[:,None] is a column vec, broadcast along rows
    weighted_pwr = tweight[:,None] * (pweight * np.power(10, arr / 10))      # Mult by weight (as col vec)
    # Total relative power
    pwr = np.sum(weighted_pwr)
    # Area of selected region
    area = np.sum(tweight) * np.sum(pweight)

    return pwr, area


# Same as above, for RP data that covers full 360 degrees of azimuth
# Phi (azimuth) values:   0 <= phi < 360
@njit
def integrate_power_density_360(arr, thetal=0.0, thetah=90.0, philow=0.0, phihigh=360.0):
    assert arr.shape[0] in [91, 46, 31, 19, 16, 11, 10, 7, 6, 4, 3, 2]
    assert arr.shape[1] in [360,180,120,90,72,60,45,40,36,30,24,20,18,15,12,10,9,8,6,5,4]
    thinc = 90 // (arr.shape[0] - 1)            # Grid increment (theta)
    phinc = 360 // (arr.shape[1])               # Grid increment (phi)
    thgrid = np.arange(0,91,thinc)              # e.g. [0, 10, ..., 90] for grid = 10
    phgrid = np.arange(0,360,phinc)             #      [0, 10, ..., 350]

    # Map angles into [-wrap,+wrap) interval ('wrap' is where RP data wraps around)
    phwrapneg = - phinc / 2                     # Phi 'wrap' angle (in neg degrees)
    phwrap = 360 + phwrapneg                    # Phi 'wrap' angle (in pos degrees)
    phil = philow if philow >= phwrapneg else philow + 360
    phih = phihigh if phihigh >= phwrapneg else phihigh + 360
    phil = phil if phil < phwrap else phil - 360
    phih = phih if phih < phwrap else phih - 360

    # 'pweight' = frac of total area represented by vertical strips associated with each grid point (within phil,phih limits)
    if np.abs(phihigh-philow-360.0) < 0.0001:
        pweight = np.full_like(phgrid,phinc) / 360.0            # Special case: range 0.0 - 360.0
    elif phil > phih:
        # Phi range crosses over end of RP data
        phlimits = get_angle_ranges(phgrid, phil, phwrap)         # Angle range to integrate for each grid point
        pweight = (phlimits[1,:] - phlimits[0,:]) / 360
        phlimits = get_angle_ranges(phgrid, phwrapneg, phih)
        pweight += (phlimits[1,:] - phlimits[0,:]) / 360
    else:
        phlimits = get_angle_ranges(phgrid, phil, phih)           # Angle range to integrate for each grid point
        pweight = (phlimits[1,:] - phlimits[0,:]) / 360

    # 'tweight' = areas of horizontal strips associated with each grid point (within thetal,thetah limits)
    thlimits = np.cos(np.deg2rad(get_angle_ranges(thgrid, thetal, thetah)))         # Angle range to integrate for each grid point
    tweight = 2 * np.pi * (thlimits[0,:] - thlimits[1,:])

    # Sum area and power weighted by area
    #   Convert dB power array to ratio, mult by corresponding area 
    #    -> pweight broadcast along cols, weight[:,None] is a column vec, broadcast along rows
    weighted_pwr = tweight[:,None] * (pweight * np.power(10, arr / 10))      # Mult by weight (as col vec)
    # Total relative power
    pwr = np.sum(weighted_pwr)
    # Area of selected region
    area = np.sum(tweight) * np.sum(pweight)

    return pwr, area




In [6]:
%%timeit
integrate_power_density_360(arr360, thetal=12, thetah=90.0, philow=0.0, phihigh=20.0)
# My exec time = 55 us/simulation

55.1 µs ± 6.29 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [2]:
# Test matching components - numpy matrix math vs numba
from numba import njit

xl_max = 200
x_m = np.arange(1,xl_max+1,dtype=float)             # Matching reactance values considered, e.g. 1,2,3...200 ohms
f_min = 28.0                # Band of interest
f_max = 29.0
f_center = np.mean([f_min,f_max])
f_num = 9                   # Num of freqs evaluated within band
z0 = 50          
# Pre-calculate 1/Xl for range of matching inductive reactances considered
freqs = np.linspace(f_min, f_max, num=f_num)        # Freqs used in feedpoint z simulation
# Array of (1 / matching reactance) at each freq; shape = (len(x_m), len(freqs))
Xlmr = np.reciprocal(((1.0j)*x_m)[:,None] @ (freqs[None,:] / f_center ))         # L

yagi = '\n'.join(
    ['CE 3-el Yagi',               
     'GW 1 20 -0.91 -2.79 0 -0.91 2.79 0 0.00635',          # R
     'GW 2 20 0 -2.56 0 0 2.56 0 0.00635',                  # DE
     'GW 3 20 1.37 -2.44 0 1.37 2.44 0 0.00635',            # D
     'GE 0 0',                                              # End of geometry; no ground plane specified
     'EX 4 2 10 2 1.0 0.0',                                 # Excitation: current source (1A), tag=2, segment=10, far end=2
     f'FR 0 {f_num} 0 0 {f_min} {(f_max-f_min)/(f_num-1)}',                               # Frequencies for XQ card: 28.0 - 29.0 MHz
     'XQ 0',                                                # Simulate feedpoint impedance
     'EN\n'                                                 # End
    ])
res = nec5_sim_stdio3([yagi]*100)

# This extracts the impedances at the 9 freqs we specified, for the 100 designs we simulated  -->  shape is (100,9)
zarr = np.array([tuple(zip(*r[0][0]))[1] for r in res])


# *** Using numpy matrix math:

# 
# Find optimum matching reactance and resulting max VSWR within band of interest
# 
#  Args:    x        complex z for each frequency in simulation (row vector)
#
#  Returns:     [matching inductive reactance, max vswr in band]  (row vector)
#
def find_opt_match(x):    
    rzs = np.reciprocal(x)                              # 1/z
    zmatch = np.reciprocal(Xlmr + rzs)                  # 1 / ( 1/z + 1/Xl ) Note: all rows of Xlmr 
    abs_refl_coef = np.abs((zmatch-z0) / (zmatch+z0))     # Reflection coefs
    vswr_curves = (1 + abs_refl_coef) / (1 - abs_refl_coef)         # vswr
    max_vswr = np.max(vswr_curves, axis=1)          # Max vswr for any row (reactance value)
    idx = np.argmin(max_vswr)                       # Index of min vswr value
    # Returns:  [0]     optimum matching inductive reactance
    #           [1]     max vswr within band of interest
    #           [2:]    vswr at each freq in band (# of points = f_num = 9 in this example)
    return np.hstack([x_m[idx], max_vswr[idx], vswr_curves[idx]])


rx_1 = 1 / ((1.0j) * (freqs / f_center ))       # Recip of +1j at each freq of interest
@njit
def find_opt_match_numba(z):    
    xl_min = 0
    vswr_min = 9999.0
    for xl in range(1,xl_max+1):
        zp = 1 / (rx_1 / xl + 1/z)              # Parallel z of feedpoint and matching Xl at each freq
        abs_refl_coef = np.abs((zp-z0) / (zp+z0))     # Reflection coefs
        vswr_curves = (1 + abs_refl_coef) / (1 - abs_refl_coef)         # vswr
        max_vswr = np.max(vswr_curves) 
        if max_vswr < vswr_min:
            vswr_min = max_vswr
            xl_min = xl
    ret = np.empty((len(z)+2))
    ret[0] = xl_min
    ret[1] = vswr_min
    ret[2:] = vswr_curves
    return ret


def find_opt_match_null(x):    
    return x


In [69]:
# %%timeit
# xl_mv = np.apply_along_axis(find_opt_match, 1, zarr)
xl_mv = np.apply_along_axis(find_opt_match_numba, 1, zarr)
# xl_mv = np.apply_along_axis(find_opt_match_null, 1, zarr)

# Original (non-numba):
# My exec time = 20 ms --> 0.2 ms/simulation

# New numba version: convert_rp_array()
# My exec time = 24.1 ms --> 0.241 ms/simulation


In [26]:

# Perform impedance transformation using a 0-180 degree transmission line

f_min = 28.0                                # Band of interest (MHz)
f_max = 29.0
f_center = np.mean([f_min,f_max])
f_num = 9                                   # Num of freqs evaluated within band
freqs = np.linspace(f_min, f_max, num=f_num)        # Freqs evaluated within band

N_DEG = 180
z0 = 50                                     # TL char impedance
z0m = 300                                   # Matching section char impedance
degs = np.linspace(0,np.pi,N_DEG)           # 0-179 degrees (in radians)
# Precompute 
cosh = np.cosh((degs * 1.0j)[:,None] @ (freqs[None,:] / f_center ))         # (degs * 1.0j)[:,None] -->  (angle*j) as a column vector, 
sinh = np.sinh((degs * 1.0j)[:,None] @ (freqs[None,:] / f_center ))         #    freqs[None,:]  -->  frequencies as a row vector

zl = zarr[0][None,:]                        # Feedpoint z at each freq as a row vector
z50 = np.full((1,f_num), 50)

In [27]:
# %%timeit
# Results, shape = (#degree increments, #freqs)
# zin = z0m * (cosh @ zl + sinh * z0m) / (sinh @ zl + cosh * z0m)        # Execution time approx 80us
zin = z0m * (cosh * zl + sinh * z0m) / (sinh * zl + cosh * z0m)
abs_refl_coef = np.absolute((zin-z0) / (zin+z0))                         # Reflection coefs
vswr_curves = (1 + abs_refl_coef) / (1 - abs_refl_coef)             # vswr
max_vswr = np.max(vswr_curves, axis=1)                              # Max vswr at each Tl length

# My exec time = 175us